In [83]:
# Connect to PostgreSQL
import psycopg2
import pandas as pd
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="heart_dwh",
    user="postgres",
    password="A1s2d3f4"
)

# Load table into DataFrame
df = pd.read_sql_query("SELECT * FROM train_data", conn)

# Close the connection
conn.close()

# Preview data
df.head()


C:\Users\Asus\AppData\Local\Temp\ipykernel_77700\3159383215.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM train_data", conn)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [91]:

question_counts = df.apply(lambda row: (row == '?').sum(), axis=1)

df = df[question_counts <= 2]


In [92]:
import numpy as np
def clean_heart_data(df):
    df.replace('?', np.nan, inplace=True)
    df['oldpeak'] = df['oldpeak'].astype(float)
    for col in df.columns:
        if col != 'oldpeak':
            df[col] = df[col].astype(float)
    df.fillna(df.median(numeric_only=True), inplace=True)
    for col in df.columns:
        if col != 'oldpeak':
            df[col] = df[col].astype(int)
    
    return df
clean_heart_data(df)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,52,1,4,140,266,0,0,134,1,2.0,2,0,6,1
916,54,0,3,130,294,0,1,100,1,0.0,2,0,6,1
917,56,1,4,155,342,1,0,150,1,3.0,2,0,6,1
918,58,0,2,180,393,0,0,110,1,1.0,2,0,7,1


In [93]:
def remove_outliers_iqr(df, factor=1.5):
    """Remove rows with outliers using IQR method for each column."""
    clean_df = df.copy()
    numeric_cols = clean_df.select_dtypes(include=['int64', 'float64']).columns

    for col in numeric_cols:
        Q1 = clean_df[col].quantile(0.18)
        Q3 = clean_df[col].quantile(0.84)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR

        # Filter out outliers
        clean_df = clean_df[(clean_df[col] >= lower_bound) & (clean_df[col] <= upper_bound)]

    return clean_df
clean_df=remove_outliers_iqr(df)

In [97]:
clean_df['target'] = clean_df['target'].apply(lambda x: 1 if x > 1 else x)
clean_df["target"].value_counts()

target
1    335
0    219
Name: count, dtype: int64

In [99]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = clean_df.drop('target', axis=1)
y = clean_df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1200)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


✅ Accuracy: 0.8918918918918919
📊 Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.85      0.87        47
           1       0.89      0.92      0.91        64

    accuracy                           0.89       111
   macro avg       0.89      0.89      0.89       111
weighted avg       0.89      0.89      0.89       111

🧩 Confusion Matrix:
 [[40  7]
 [ 5 59]]


In [100]:
import pickle

with open("model.pickle", "wb") as f:
    pickle.dump(model, f)
